# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [30]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [31]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,weno,7.4515,151.8468,28.97,84,100,4.12,FM,1722041451
1,1,zhangjiakou,40.8100,114.8794,24.88,76,83,0.81,CN,1722041452
2,2,blackmans bay,-43.0167,147.3167,10.41,70,100,1.96,AU,1722041453
3,3,portland,45.5234,-122.6762,29.38,35,20,6.17,US,1722040980
4,4,kalmunai,7.4167,81.8167,28.27,61,100,2.38,LK,1722041455


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [32]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [33]:
# Narrow down cities that fit criteria and drop any results with null values


criteria1 = (city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) #Criteria 1: Max Temp between 21 and 27
criteria2 = city_data_df["Wind Speed"] < 4.5  # Criteria 2: Wind Speed less than 4.5 m/s
criteria3 = city_data_df["Cloudiness"] == 0  #Criteria 3: Cloudiness = 0

combined_criteria = criteria1 & criteria2 & criteria3

filtered_city_data_df = city_data_df[combined_criteria]

# Drop any rows with null values
cleaned_city_data_df = filtered_city_data_df.dropna()

# Display sample data
cleaned_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,jamestown,42.0970,-79.2353,21.80,60,0,3.09,US,1722041474
21,21,albany,42.6001,-73.9662,21.07,63,0,3.88,US,1722041402
38,38,bethel,41.3712,-73.4140,23.83,71,0,0.00,US,1722041372
40,40,sandpoint,48.2766,-116.5533,26.92,28,0,3.09,US,1722041500
98,98,mizdah,31.4451,12.9801,23.27,57,0,3.28,LY,1722041566
126,126,hamilton,39.1834,-84.5333,25.69,52,0,3.09,US,1722041204
138,138,reyes,-14.3167,-67.3833,24.52,57,0,0.45,BO,1722041564
178,178,belyy yar,53.6039,91.3903,23.94,78,0,4.00,RU,1722041665
292,292,barra de santo antonio,-9.4047,-35.5072,22.60,79,0,4.36,BR,1722041805
305,305,florence,43.7667,11.2500,24.90,73,0,2.57,IT,1722041660


### Step 3: Create a new DataFrame called `hotel_df`.

In [34]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_city_data_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]
hotel_df.head()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
19,jamestown,US,42.0970,-79.2353,60,
21,albany,US,42.6001,-73.9662,63,
38,bethel,US,41.3712,-73.4140,71,
40,sandpoint,US,48.2766,-116.5533,28,
98,mizdah,LY,31.4451,12.9801,57,
126,hamilton,US,39.1834,-84.5333,52,
138,reyes,BO,-14.3167,-67.3833,57,
178,belyy yar,RU,53.6039,91.3903,78,
292,barra de santo antonio,BR,-9.4047,-35.5072,79,
305,florence,IT,43.7667,11.2500,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [35]:
# Set parameters to search for a hotel
limit = 20
radius = 10000
filters = f"circle:{"Lng"},{"Lat"},{radius}"
bias = f"proximity:{"Lng"},{"Lat"}"

params = {
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = # YOUR CODE HERE

    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

SyntaxError: invalid syntax (3677125095.py, line 22)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)